In [5]:
from scm_irl.env.scm_irl_env import ScmIrlEnv
from sllib.conversions.geo_conversions import north_east_to_lat_lon, mps2knots, lat_lon_to_north_east
import numpy as np
import wandb
from wandb.integration.sb3 import WandbCallback
from hydra.experimental import initialize_config_dir, compose
from scm_irl.utils.process_scenario import Scenario
from scm_irl.utils.env_wrappers import FlatObservationWrapper, ResNetObservationWrapper

import gymnasium as gym
# Check if the environment is registered

gym.register(
    id='ScmIrl-v0',
    entry_point='scm_irl.env:ScmIrlEnv',
    max_episode_steps=1000,
)

with initialize_config_dir(config_dir="/home/rolando/Documents/DTU/SCMarine/scm_irl/scm_irl/conf"):
    cfg = compose(config_name="train_rl.yaml")


path = "/home/rolando/Documents/DTU/SCMarine/ais-and-charts-sample/ais_data/scenario_2a66ceaf61"

scenario = Scenario(cfg, path)


env = ScmIrlEnv(cfg, scenario, mmsi=215811000, awareness_zone = [200, 600, 200, 200], render_mode="rgb_array")

print(env.observation_space)


env = ResNetObservationWrapper(env)
env = FlatObservationWrapper(env)

timestep = env.timestep
action = env.get_action_from_vessel(timestep)
print(f"action: {action}")
#action = (action[0], -0.5)
agent_state, reward, terminate, truncated, info = env.step(action)

# for key in agent_state.keys():
#     print(f"{key}: {agent_state[key].shape}")
print(agent_state.shape)

#env = ResNetObservationWrapper(env)

print(env.observation_space)


#env = ScmIrlEnv(path, mmsi=215811000, awareness_zone = [200, 500, 200, 200], start_time_reference=1577905000.0)

#env.reset()

#env.step(np.array([1.0,1.0]))

/home/rolando/anaconda3/envs/scm/lib/python3.11/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment ScmIrl-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/rolando/anaconda3/envs/scm/lib/python3.11/site-packages/hydra/experimental/initialize.py:116: UserWarning: hydra.experimental.initialize_config_dir() is no longer experimental. Use hydra.initialize_config_dir().
  deprecation_warning(message=message)
/home/rolando/anaconda3/envs/scm/lib/python3.11/site-packages/hydra/experimental/initialize.py:118: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize_config_dir(
/home/rolando/anaconda3/envs/scm/lib/python3.11/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_

mmsi: 215811000, start_time: 1577904600.0, end_time: 1577906080.0
Dict('agent_state': Box(-inf, inf, (4,), float32), 'observation_matrix': Box(0.0, 255.0, (400, 800, 3), float32), 'vessel_params': Box(-inf, inf, (5,), float32))


/home/rolando/anaconda3/envs/scm/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rolando/anaconda3/envs/scm/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/rolando/anaconda3/envs/scm/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.timestep to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.timestep` for environment variables or `env.get_wrapper_attr('timestep')` that will searc

action: (0.12288100909090911, 0.06671717171717151)
(166409,)
Box(-inf, inf, (166409,), float32)


In [2]:
import gymnasium as gym
import wandb
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

def make_env():
    #env = gym.make("CartPole-v1", render_mode="rgb_array")
    path = "/home/rolando/Documents/DTU/SCMarine/ais-and-charts-sample/ais_data/scenario_2a66ceaf61"
    env = ScmIrlEnv(path, mmsi=215811000, awareness_zone = [200, 500, 200, 200], start_time_reference=1577905000.0, render_mode="rgb_array")
    env = gym.wrappers.RecordVideo(env, f"videos")  # record videos
    env = gym.wrappers.RecordEpisodeStatistics(env)  # record stats such as returns
    return env

config = {
    "policy": 'MultiInputPolicy',
    "total_timesteps": 100000,
    "env_name": "ScmIrl-v0",
}

wandb.init(
    config=config,
    sync_tensorboard=True,  # automatically upload SB3's tensorboard metrics to W&B
    project="scmirl",
    monitor_gym=True,       # automatically upload gym environements' videos
    save_code=True,
)

env = DummyVecEnv([make_env])
model = PPO(config['policy'], env, verbose=1, tensorboard_log=f"runs/ppo")
model.learn(total_timesteps=config['total_timesteps'], progress_bar=True)
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rolando-esq (dtu-projects). Use `wandb login --relogin` to force relogin


mmsi: 215811000, start_time: 1577905000.0, end_time: 1577906100.0


/home/rolando/anaconda3/envs/scm/lib/python3.11/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/rolando/Documents/DTU/SCMarine/scm_irl/notebooks/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Using cuda device
Logging to runs/ppo/PPO_5


Output()

...

File "/home/rolando/anaconda3/envs/scm/lib/python3.11/site-packages/IPython/core/display_functions.py", line 93, 
in publish_display_data

display_pub.publish(

File "/home/rolando/anaconda3/envs/scm/lib/python3.11/site-packages/wandb/sdk/wandb_init.py", line 477, in 
publish

ipython.display_pub._orig_publish(data, metadata=metadata, **kwargs)

File "/home/rolando/anaconda3/envs/scm/lib/python3.11/site-packages/ipykernel/zmqshell.py", line 103, in publish

In [2]:
# import stablebaselines3

config = {
    "policy_type": "MultiInputPolicy",
    "total_timesteps": 100000,
    "env_name": "ScmIrl-v0",
}

run = wandb.init(
    project="scmirl",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
)


from stable_baselines3 import PPO

model = PPO("MultiInputPolicy", env, verbose = 1)

model.learn(total_timesteps=100000, 
            callback=WandbCallback(
                gradient_save_freq=100,
                model_save_path=f"models/{run.id}",
                verbose=2,),
            log_interval=10,
            tb_log_name="ppo",
            reset_num_timesteps=False,
            progress_bar=True,
                )

model.save("ppo_ais")

run.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: rolando-esq (dtu-projects). Use `wandb login --relogin` to force relogin


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Output()

agent out of the scenario

...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 110          |
|    ep_rew_mean          | -3.22e+04    |
| time/                   |              |
|    fps                  | 4            |
|    iterations           | 20           |
|    time_elapsed         | 9894         |
|    total_timesteps      | 40960        |
| train/                  |              |
|    approx_kl            | 0.0023366704 |
|    clip_fraction        | 0.00083      |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.8         |
|    explained_variance   | -1.19e-07    |
|    learning_rate        | 0.0003       |
|    loss                 | 1.46e+07     |
|    n_updates            | 190          |
|    policy_gradient_loss | -0.000678    |
|    std                  | 0.982        |
|    value_loss           | 2.53e+07     |
------------------------------------------


agent out of the scenario

agent out of the scenario

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 110         |
|    ep_rew_mean          | -3.15e+04   |
| time/                   |             |
|    fps                  | 5           |
|    iterations           | 30          |
|    time_elapsed         | 11211       |
|    total_timesteps      | 61440       |
| train/                  |             |
|    approx_kl            | 0.003657495 |
|    clip_fraction        | 0.0084      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.79       |
|    explained_variance   | 0           |
|    learning_rate        | 0.0003      |
|    loss                 | 1.01e+07    |
|    n_updates            | 290         |
|    policy_gradient_loss | -0.00151    |
|    std                  | 0.975       |
|    value_loss           | 2.29e+07    |
-----------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 110          |
|    ep_rew_mean          | -3.13e+04    |
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 40           |
|    time_elapsed         | 12898        |
|    total_timesteps      | 81920        |
| train/                  |              |
|    approx_kl            | 0.0037992706 |
|    clip_fraction        | 0.0106       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.82        |
|    explained_variance   | 5.96e-08     |
|    learning_rate        | 0.0003       |
|    loss                 | 1.12e+07     |
|    n_updates            | 390          |
|    policy_gradient_loss | -0.00209     |
|    std                  | 0.99         |
|    value_loss           | 2.2e+07      |
------------------------------------------
